# Imports

In [1]:
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import numpy as np

%matplotlib inline

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
device

device(type='cpu')

Loading data files
==================




In [3]:
%%capture
!wget https://download.pytorch.org/tutorial/data.zip
!unzip -o data.zip

# Vocabulary Class (and Text Preprocessing)

In [4]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "<SOS>", 1: "<EOS>"}
        self.n_words = 2  # Count SOS and EOS
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1


# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs


MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]


def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [5]:
input_lang, output_lang, pairs = prepareData('eng', 'fra', True)

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4345
eng 2803


# Random Sample for Subsequent Runs

A random sample from the 'pairs' list was chosen so that the maximum length of the input and output sequences do not change upon subsequent runs, since we are not allowed to use a for-loop in our code!

In [6]:
# sample = random.choice(pairs)
sample = ['vous me faites rougir .', 'you re making me blush .']
sample

['vous me faites rougir .', 'you re making me blush .']

In [7]:
input_sentence = sample[0]
output_sentence = sample[1]

In [8]:
input_indices = [input_lang.word2index[word] for word in input_sentence.split(' ')]
target_indices = [output_lang.word2index[word] for word in output_sentence.split(' ')]

In [9]:
input_indices.append(EOS_token)
target_indices.append(EOS_token)

In [10]:
input_indices, target_indices

([118, 27, 590, 2795, 5, 1], [129, 78, 505, 343, 1655, 4, 1])

In [11]:
input_tensor = torch.tensor(input_indices, dtype=torch.long, device = device)
output_tensor = torch.tensor(target_indices, dtype=torch.long, device = device)

In [12]:
input_tensor.shape, output_tensor.shape

(torch.Size([6]), torch.Size([7]))

# Dimensions

In [13]:
DIM_IN = input_lang.n_words
DIM_OUT = output_lang.n_words
DIM_HID = 256 # arbitraily chosen! must be same for encoder and decoder!
MAX_LEN_IN = input_tensor.size()[0] # length of the input sequence under consideration
MAX_LEN_OUT = output_tensor.size()[0] # length of the output sequence under consideration

In [14]:
DIM_IN, DIM_OUT, DIM_HID, MAX_LEN_IN, MAX_LEN_OUT

(4345, 2803, 256, 6, 7)

# Encoder

## Instantiating layers

In [15]:
embedding = nn.Embedding(DIM_IN, DIM_HID).to(device)
lstm = nn.LSTM(DIM_HID, DIM_HID).to(device)

## Feeding Input Sequence to Encoder

In [16]:
encoder_outputs = torch.zeros(MAX_LEN_IN, DIM_HID, device=device) # array to store outputs of enocders so that it can be used for attention
hidden = torch.zeros(1, 1, DIM_HID, device=device) # first hidden state initialized as zeros
cell = torch.zeros(1, 1, DIM_HID, device=device) # first hidden state initialized as zeros

In [17]:
input = input_tensor[0].view(-1, 1)
embedded_input = embedding(input)
output, (hidden, cell) = lstm(embedded_input, (hidden, cell))
encoder_outputs[0] += output[0,0]

In [18]:
input = input_tensor[1].view(-1, 1)
embedded_input = embedding(input)
output, (hidden, cell) = lstm(embedded_input, (hidden, cell))
encoder_outputs[1] += output[0,0]

In [19]:
input = input_tensor[2].view(-1, 1)
embedded_input = embedding(input)
output, (hidden, cell) = lstm(embedded_input, (hidden, cell))
encoder_outputs[2] += output[0,0]

In [20]:
input = input_tensor[3].view(-1, 1)
embedded_input = embedding(input)
output, (hidden, cell) = lstm(embedded_input, (hidden, cell))
encoder_outputs[3] += output[0,0]

In [21]:
input = input_tensor[4].view(-1, 1)
embedded_input = embedding(input)
output, (hidden, cell) = lstm(embedded_input, (hidden, cell))
encoder_outputs[4] += output[0,0]

In [22]:
input = input_tensor[5].view(-1, 1)
embedded_input = embedding(input)
output, (hidden, cell) = lstm(embedded_input, (hidden, cell))
encoder_outputs[5] += output[0,0]

# Decoder

## Instantiating Layers

In [23]:
embedding = nn.Embedding(DIM_OUT, DIM_HID).to(device)
attn = nn.Linear(DIM_HID, DIM_HID)
lstm_inp = nn.Linear(DIM_HID * 2, DIM_HID).to(device) #this layer takes care of the mismatched dimensions
lstm = nn.LSTM(DIM_HID, DIM_HID).to(device)
linear_out = nn.Linear(DIM_HID*2, DIM_OUT).to(device)

In [24]:
predicted_sentence = []

## Feeding to the Decoder - Word 1

In [25]:
decoder_input = torch.tensor([[SOS_token]], device=device) # We start from the <SOS> Token
decoder_hidden = hidden # what we got from the output of the encoder from the last word
decoder_cell = cell # what we got from the output of the encoder from the last word

embedded = embedding(decoder_input)

## Attn module
attn_energies = torch.zeros(MAX_LEN_IN).to(device)
for i in range(MAX_LEN_IN):
  energy = attn(encoder_outputs[i])
  attn_energies[i] = hidden[0,0].dot(energy) + cell[0,0].dot(energy)
attn_weights = F.softmax(attn_energies, dim=0).unsqueeze(0).unsqueeze(0)
##

context = attn_weights.bmm(encoder_outputs.unsqueeze(1).transpose(0, 1))

input_to_lstm1 = torch.cat((embedded, context), 2)
input_to_lstm2 = lstm_inp(input_to_lstm1)
output, (decoder_hidden, decoder_cell) = lstm(input_to_lstm2, (decoder_hidden, decoder_cell))

output = F.log_softmax(linear_out(torch.cat((output, context), 2)), dim=2)
top_value, top_index = output.data.topk(1) # same as using np.argmax

out_word = output_lang.index2word[top_index.item()]
print(out_word)
predicted_sentence.append(out_word)

chemistry


## Feeding to the Decoder - Word 2

In [26]:
teacher_forcing_ratio = 0.5
use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

if use_teacher_forcing:
  decoder_input = torch.tensor([[target_indices[0]]], device=device)
else:
  decoder_input = torch.tensor([[top_index.item()]], device=device)


In [27]:
embedded = embedding(decoder_input)

## Attn module
attn_energies = torch.zeros(MAX_LEN_IN).to(device)
for i in range(MAX_LEN_IN):
  energy = attn(encoder_outputs[i])
  attn_energies[i] = hidden[0,0].dot(energy) + cell[0,0].dot(energy)
attn_weights = F.softmax(attn_energies, dim=0).unsqueeze(0).unsqueeze(0)
##

context = attn_weights.bmm(encoder_outputs.unsqueeze(1).transpose(0, 1))

input_to_lstm1 = torch.cat((embedded, context), 2)
input_to_lstm2 = lstm_inp(input_to_lstm1)
output, (decoder_hidden, decoder_cell) = lstm(input_to_lstm2, (decoder_hidden, decoder_cell))

output = F.log_softmax(linear_out(torch.cat((output, context), 2)), dim=2)
top_value, top_index = output.data.topk(1) # same as using np.argmax

out_word = output_lang.index2word[top_index.item()]
print(out_word)
predicted_sentence.append(out_word)

chemistry


## Feeding to the Decoder - Word 3

In [28]:
teacher_forcing_ratio = 0.5
use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

if use_teacher_forcing:
  decoder_input = torch.tensor([[target_indices[0]]], device=device)
else:
  decoder_input = torch.tensor([[top_index.item()]], device=device)


In [29]:
embedded = embedding(decoder_input)

## Attn module
attn_energies = torch.zeros(MAX_LEN_IN).to(device)
for i in range(MAX_LEN_IN):
  energy = attn(encoder_outputs[i])
  attn_energies[i] = hidden[0,0].dot(energy) + cell[0,0].dot(energy)
attn_weights = F.softmax(attn_energies, dim=0).unsqueeze(0).unsqueeze(0)
##

context = attn_weights.bmm(encoder_outputs.unsqueeze(1).transpose(0, 1))

input_to_lstm1 = torch.cat((embedded, context), 2)
input_to_lstm2 = lstm_inp(input_to_lstm1)
output, (decoder_hidden, decoder_cell) = lstm(input_to_lstm2, (decoder_hidden, decoder_cell))

output = F.log_softmax(linear_out(torch.cat((output, context), 2)), dim=2)
top_value, top_index = output.data.topk(1) # same as using np.argmax

out_word = output_lang.index2word[top_index.item()]
print(out_word)
predicted_sentence.append(out_word)

been


## Feeding to the Decoder - Word 4

In [30]:
teacher_forcing_ratio = 0.5
use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

if use_teacher_forcing:
  decoder_input = torch.tensor([[target_indices[0]]], device=device)
else:
  decoder_input = torch.tensor([[top_index.item()]], device=device)


In [31]:
embedded = embedding(decoder_input)

## Attn module
attn_energies = torch.zeros(MAX_LEN_IN).to(device)
for i in range(MAX_LEN_IN):
  energy = attn(encoder_outputs[i])
  attn_energies[i] = hidden[0,0].dot(energy) + cell[0,0].dot(energy)
attn_weights = F.softmax(attn_energies, dim=0).unsqueeze(0).unsqueeze(0)
##

context = attn_weights.bmm(encoder_outputs.unsqueeze(1).transpose(0, 1))

input_to_lstm1 = torch.cat((embedded, context), 2)
input_to_lstm2 = lstm_inp(input_to_lstm1)
output, (decoder_hidden, decoder_cell) = lstm(input_to_lstm2, (decoder_hidden, decoder_cell))

output = F.log_softmax(linear_out(torch.cat((output, context), 2)), dim=2)
top_value, top_index = output.data.topk(1) # same as using np.argmax

out_word = output_lang.index2word[top_index.item()]
print(out_word)
predicted_sentence.append(out_word)

cantankerous


## Feeding to the Decoder - Word 5

In [32]:
teacher_forcing_ratio = 0.5
use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

if use_teacher_forcing:
  decoder_input = torch.tensor([[target_indices[0]]], device=device)
else:
  decoder_input = torch.tensor([[top_index.item()]], device=device)


In [33]:
embedded = embedding(decoder_input)

## Attn module
attn_energies = torch.zeros(MAX_LEN_IN).to(device)
for i in range(MAX_LEN_IN):
  energy = attn(encoder_outputs[i])
  attn_energies[i] = hidden[0,0].dot(energy) + cell[0,0].dot(energy)
attn_weights = F.softmax(attn_energies, dim=0).unsqueeze(0).unsqueeze(0)
##

context = attn_weights.bmm(encoder_outputs.unsqueeze(1).transpose(0, 1))

input_to_lstm1 = torch.cat((embedded, context), 2)
input_to_lstm2 = lstm_inp(input_to_lstm1)
output, (decoder_hidden, decoder_cell) = lstm(input_to_lstm2, (decoder_hidden, decoder_cell))

output = F.log_softmax(linear_out(torch.cat((output, context), 2)), dim=2)
top_value, top_index = output.data.topk(1) # same as using np.argmax

out_word = output_lang.index2word[top_index.item()]
print(out_word)
predicted_sentence.append(out_word)

pressing


## Feeding to the Decoder - Word 6

In [34]:
teacher_forcing_ratio = 0.5
use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

if use_teacher_forcing:
  decoder_input = torch.tensor([[target_indices[0]]], device=device)
else:
  decoder_input = torch.tensor([[top_index.item()]], device=device)


In [35]:
embedded = embedding(decoder_input)

## Attn module
attn_energies = torch.zeros(MAX_LEN_IN).to(device)
for i in range(MAX_LEN_IN):
  energy = attn(encoder_outputs[i])
  attn_energies[i] = hidden[0,0].dot(energy) + cell[0,0].dot(energy)
attn_weights = F.softmax(attn_energies, dim=0).unsqueeze(0).unsqueeze(0)
##

context = attn_weights.bmm(encoder_outputs.unsqueeze(1).transpose(0, 1))

input_to_lstm1 = torch.cat((embedded, context), 2)
input_to_lstm2 = lstm_inp(input_to_lstm1)
output, (decoder_hidden, decoder_cell) = lstm(input_to_lstm2, (decoder_hidden, decoder_cell))

output = F.log_softmax(linear_out(torch.cat((output, context), 2)), dim=2)
top_value, top_index = output.data.topk(1) # same as using np.argmax

out_word = output_lang.index2word[top_index.item()]
print(out_word)
predicted_sentence.append(out_word)

pressing


## Feeding to the Decoder - Word 7

In [36]:
teacher_forcing_ratio = 0.5
use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

if use_teacher_forcing:
  decoder_input = torch.tensor([[target_indices[0]]], device=device)
else:
  decoder_input = torch.tensor([[top_index.item()]], device=device)


In [37]:
embedded = embedding(decoder_input)

## Attn module
attn_energies = torch.zeros(MAX_LEN_IN).to(device)
for i in range(MAX_LEN_IN):
  energy = attn(encoder_outputs[i])
  attn_energies[i] = hidden[0,0].dot(energy) + cell[0,0].dot(energy)
attn_weights = F.softmax(attn_energies, dim=0).unsqueeze(0).unsqueeze(0)
##

context = attn_weights.bmm(encoder_outputs.unsqueeze(1).transpose(0, 1))

input_to_lstm1 = torch.cat((embedded, context), 2)
input_to_lstm2 = lstm_inp(input_to_lstm1)
output, (decoder_hidden, decoder_cell) = lstm(input_to_lstm2, (decoder_hidden, decoder_cell))

output = F.log_softmax(linear_out(torch.cat((output, context), 2)), dim=2)
top_value, top_index = output.data.topk(1) # same as using np.argmax

out_word = output_lang.index2word[top_index.item()]
print(out_word)
predicted_sentence.append(out_word)

reluctant


In [38]:
predicted_sentence = ' '.join(predicted_sentence)
predicted_sentence

'chemistry chemistry been cantankerous pressing pressing reluctant'